In [ ]:
!pip3 install mesa

In [ ]:
# La clase `Model` se hace cargo de los atributos a nivel del modelo, maneja los agentes. 
# Cada modelo puede contener múltiples agentes y todos ellos son instancias de la clase `Agent`.
from mesa import Agent, Model 

# Debido a que necesitamos varios agentes por celda elegimos `MultiGrid` que permite tener varios agentes por celda.
from mesa.space import MultiGrid

# Con `SimultaneousActivation` hacemos que todos los agentes se activen de manera simultanea.
from mesa.time import SimultaneousActivation

# Vamos a hacer uso de `DataCollector` para obtener el grid completo cada paso (o generación) y lo usaremos para graficarlo.
from mesa.datacollection import DataCollector

# mathplotlib lo usamos para graficar/visualizar como cambia el modelo.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Definimos los siguientes paquetes para manejar valores númericos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

#Definimos variable global
movimiento_total = 0


In [ ]:
def get_grid(model):
    '''
    Esta es una función auxiliar que nos permite guardar el grid para cada uno de los agentes.
    '''
    grid = np.zeros((model.grid.width, model.grid.height))
    for cell in model.grid.coord_iter():
        cell_content, x, y = cell
        num_cajas = 0
        for content in cell_content:
            if content.tipo == 6:
                grid[x][y] = 6
                break

            elif content.cant_cajas >= 1:
                num_cajas = content.cant_cajas
                grid[x][y] = num_cajas

    return grid

class Robot(Agent):
    '''
    Define una clase Robot que se va a encargar de mover las cajas hasta que estén acomodadas
    '''

    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.tipo = 6
        self.estado = 0

    def poner_caja(self):
        celda = self.model.grid.get_cell_list_contents([self.pos])
        esta_celda = [obj for obj in celda if isinstance(obj, Piso)]
        esta_celda[0].cant_cajas = esta_celda[0].cant_cajas + 1
        self.estado = 0

    def mover_derecha(self):
      [(x, y)] = [self.pos]
      y = y+1
      self.model.grid.move_agent(self,(x, y))
      celda = self.model.grid.get_cell_list_contents([self.pos])
      esta_celda = [obj for obj in celda if isinstance(obj, Piso)]
      if esta_celda[0].cant_cajas < 5:
        self.poner_caja()
      else:
          self.mover_derecha()
          self.poner_caja()
      
    
            
    def corner_move(self):
        [(x, y)] = [self.pos]
        if x>0:
            x = x-1
        if y>0:
            y = y-1
        self.model.grid.move_agent(self, (x, y))

        if x == 0 and y == 0:
            celda = self.model.grid.get_cell_list_contents([self.pos])
            esta_celda = [obj for obj in celda if isinstance(obj, Piso)]
            if esta_celda[0].cant_cajas < 5:
              self.poner_caja()
            else:
                self.mover_derecha()
            

    def ran_move(self):
        posibles_movimientos = []
        vecinos = self.model.grid.get_neighborhood(self.pos, moore=False, include_center=False)
        for i in range(len(vecinos)):
            celdas = self.model.grid.get_cell_list_contents([vecinos[i]])
            for j in range(len(celdas)):
                if celdas[j].tipo != 6:
                    posibles_movimientos.append(vecinos[i])
                    next_pos = self.random.choice(posibles_movimientos)
                    self.model.grid.move_agent(self, next_pos)

    def step(self):
        if self.estado == 0:        
            celdas = self.model.grid.get_cell_list_contents([self.pos])
            esta_celda = [obj for obj in celdas if isinstance(obj, Piso)]
            [(x, y)] = [self.pos]
            if not x == 0 and not y == 0:
                if esta_celda[0].cant_cajas == 1:
                    esta_celda[0].cant_cajas = 0
                    self.estado = 1
                else:
                    self.ran_move()
            else:
              self.ran_move()
        else:
            self.corner_move()
        global movimiento_total
        movimiento_total = movimiento_total +1

class Piso(Agent):
    '''
    Representa a un agente piso con un atributo cant_cajas que inicia en 1 o 0, cuando se apilan las cajas aumenta conforme se apilan las cajas.
    '''
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.tipo = 0
        self.cant_cajas = 0

    def step(self):
        celdas = self.model.grid.get_cell_list_contents([self.pos])
        for i in range(len(celdas)):
            if(celdas[i].tipo==1):
                self.cant_cajas = 0;

class Almacen(Model):
    '''
    Define el modelo del Almacen
    '''
    def __init__(self, width, height, num_cajas):
        self.num_robots = 5
        self.num_cajas = num_cajas
        self.grid = MultiGrid(width, height, False)
        self.schedule = SimultaneousActivation(self)

        # Agregar los pisos vacios
        for (content, x, y) in self.grid.coord_iter():
            a = Piso((x, y), self)
            self.grid.place_agent(a, (x, y))
            self.schedule.add(a)

        # Agregar cajas al piso en posiciones aleatorias
        cont_cajas = 0
        while cont_cajas < self.num_cajas:
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            a = Piso(100, self)
            self.grid.place_agent(a, (x, y))
            celda = self.grid.get_cell_list_contents([a.pos])
            esta_celda = [obj for obj in celda if isinstance(obj, Piso)]
            if esta_celda[0].cant_cajas == 0 and not x == 0 and not y == 0:
                esta_celda[0].cant_cajas = 1                
                self.grid.remove_agent(a)
                cont_cajas+=1

        cont_robots = 0
        while cont_robots < 5:
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            a = Robot(cont_robots, self)
            celda = self.grid.get_cell_list_contents([(x, y)])
            esta_celda = [obj for obj in celda if isinstance(obj, Robot)]
            if len(esta_celda) == 0 and (not x == 0 and not y == 0):
                self.grid.place_agent(a, (x, y))
                self.schedule.add(a)               
                cont_robots+=1

 
        
        # Aquí definimos con colector para obtener el grid completo.
        self.datacollector = DataCollector(
            model_reporters={"Grid": get_grid})
        

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()

In [ ]:
# Definimos el tamaño del Grid
N = int(input("Ingrese valor X: "))
M = int(input("Ingrese valor Y: "))

# Definimos número de agentes que limpian
NUM_AGENTES = int(input("Ingrese cantidad de cajas: "))

#Definimos el tiempo maximo de ejecución
Tiempo_Max = float(input("Ingrese el tiempo de ejecución: "))

# Registramos el tiempo de inicio y corremos el modelo
start_time = time.time()
tiempo_inicio = str(datetime.timedelta(seconds = Tiempo_Max))
model = Almacen(N, M, NUM_AGENTES)
while((time.time() - start_time) < Tiempo_Max):
    model.step()

# Imprimimos el tiempo que le tomó correr al modelo.
print('Tiempo de ejecución:', str(datetime.timedelta(seconds=(time.time() - start_time))))

Ingrese valor X: 10
Ingrese valor Y: 10
Ingrese cantidad de cajas: 30
Ingrese el tiempo de ejecución: .07
Tiempo de ejecución: 0:00:00.070643


In [ ]:
all_grid = model.datacollector.get_model_vars_dataframe()
print("Movimientos totales: ", movimiento_total)

Movimientos totales:  6135


In [ ]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap='Greys')

def animate(i):
    patch.set_data(all_grid.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=len(all_grid))

In [ ]:
anim